In [ ]:
from sklearn.externals import joblib
from features import gen_attrs

#初始化账户
def initialize(account):
    account.security = '000300.SH'
    account.isbuy = False

def handle_data(account,data):
    time = get_last_datetime().strftime("%Y%m%d")
    log.info(time)
    #获取证券过去20日的收盘价数据
    history = get_price(account.security,'20050101',time,'1d',\
                ['open','close','low','high','avg_price','prev_close','volume'],False,'pre')
    # history = data.attribute_history(account.security, ['open','close','low','high','avg_price','prev_close','volume'], 34, '1d')   
    log.info(history)
    # 计算当天属性值
    attrs, logreturn = gen_attrs(history, 33)
    # state = 1
    model = joblib.load('AI/nn.pkl')
    scaler = joblib.load('AI/scaler.pkl')
    attrs = scaler.transform(attrs)
    state = model.predict(attrs)[-1]
    if state == 1 and account.isbuy == False:
    # if state in [1, 5, 10] and g.is_pos == 0:
        # order_value('510300.OF',account.cash)
        order_value('159919.OF',account.cash)
        # order_value('510050.OF',account.cash)
        log.info("买入 %s" % (account.security))
        account.isbuy = True
    elif state == 0 and account.isbuy == True:
    # elif state not in [1, 5, 10] and g.is_pos == 1:
        # order_target('510300.OF',0)
        order_target('159919.OF',0)
        # order_target('510050.OF',0)
        log.info("卖出 %s" % (account.security))
        account.isbuy = False